"""

Usage:
 - Fichier matches.csv requis : colonnes -> date,home_team,away_team,home_goals,away_goals,competition
 - Lancer: python Elo.ipynb
 - Sorties:
     ratings_over_time.csv  -> évolution match par match
     final_ratings.csv      -> classement final

Règles spécifiques (selon demande):
 - K identique pour toutes compétitions.
 - Initialisation:
     * National: 1200
     * L2: 1400
     * L1: 1600
 - Clubs actifs = au moins un match dans les 6 derniers mois.
 - Clubs inactifs = aucun match depuis 6 mois -> -100 pts tous les 6 mois (jusqu’à plancher 800 Elo).
 - Un club nouveau en National commence à 1200.
 - Un club revenu après sortie hors-circuit repart à 1200.

"""

In [1]:
import csv
import datetime
from collections import defaultdict, OrderedDict
import sys

In [2]:
K_FACTOR = 30
INITIAL_RATINGS = {
    'ligue-1': 1600,
    'ligue-2': 1400,
    'national': 1200,
    'default': 1200,
}
DRAW_SCORE = 0.5
WIN_SCORE = 1.0
LOSS_SCORE = 0.0

INACTIVITY_MONTHS = 9
INACTIVITY_PENALTY = 250
MIN_RATING = 800

MATCHES_CSV = 'C:\\1_Programmes Python\\Statistiques\\Classement Elo foot\\New version\\Résultats\\all_results\\resultats_1997_2026.csv'
OUT_TIMESERIES = 'C:\\1_Programmes Python\\Statistiques\\Classement Elo foot\\New version\\Elo\\ratings_over_time.csv'
OUT_FINAL = 'C:\\1_Programmes Python\\Statistiques\\Classement Elo foot\\New version\\Elo\\final_ratings.csv'

In [3]:
class EloRatings:
    def __init__(self):
        self.ratings = {}           # team -> rating
        self.last_active = {}       # team -> date
        self.history = []
        self.inactivity_penalties = {}  # team -> nombre de pénalités déjà appliquées
    

    def get_initial_rating(self, team, comp):
        if team in self.ratings:
            return self.ratings[team]
        base = INITIAL_RATINGS.get(comp, INITIAL_RATINGS['default'])
        self.ratings[team] = base
        self.inactivity_penalties[team] = 0
        return base

    def apply_global_inactivity(self, current_date):
        for team, last in list(self.last_active.items()):
            # Mois d'inactivité
            months_inactive = (current_date.year - last.year) * 12 + (current_date.month - last.month)
            penalties = months_inactive // INACTIVITY_MONTHS

            # Si le club n’a pas rejoué depuis assez longtemps
            already_applied = self.inactivity_penalties.get(team, 0)
            to_apply = int(penalties - already_applied)

            for _ in range(to_apply):
                if self.ratings[team] > MIN_RATING:
                    new_rating = max(MIN_RATING, self.ratings[team] - INACTIVITY_PENALTY)
                    row = OrderedDict()
                    row['date'] = current_date.isoformat()
                    row['competition'] = "inactivity"
                    row['home_team'] = team
                    row['away_team'] = ""
                    row['home_goals'] = ""
                    row['away_goals'] = ""
                    row['home_before'] = round(self.ratings[team], 2)
                    row['away_before'] = ""
                    row['penalty_home'] = INACTIVITY_PENALTY
                    row['penalty_away'] = ""
                    row['delta_home'] = -INACTIVITY_PENALTY
                    row['delta_away'] = ""
                    row['home_after'] = round(new_rating, 2)
                    row['away_after'] = ""
                    self.history.append(row)
                    self.ratings[team] = new_rating

            self.inactivity_penalties[team] = penalties

    def update_match(self, date, home_team, away_team, home_goals, away_goals, competition):
        # Si une équipe revient après > 6 mois → reset à 1200
        for team in [home_team, away_team]:
            last = self.last_active.get(team)
            if last:
                months_inactive = (date.year - last.year) * 12 + (date.month - last.month)
                if months_inactive >= INACTIVITY_MONTHS:
                    self.ratings[team] = 1200
                    self.inactivity_penalties[team] = 0  # reset compteur
            else:
                # première apparition
                self.get_initial_rating(team, competition)

        home_rating = self.ratings[home_team]
        away_rating = self.ratings[away_team]

        # Résultat du match
        if home_goals > away_goals:
            score_home, score_away = WIN_SCORE, LOSS_SCORE
        elif home_goals < away_goals:
            score_home, score_away = LOSS_SCORE, WIN_SCORE
        else:
            score_home = score_away = DRAW_SCORE

        # Attendu
        exp_home = 1.0 / (1.0 + 10 ** (-(home_rating - away_rating) / 400.0))
        exp_away = 1.0 - exp_home

        # ΔELO
        delta_home = K_FACTOR * (score_home - exp_home)
        delta_away = K_FACTOR * (score_away - exp_away)

        new_home = home_rating + delta_home
        new_away = away_rating + delta_away

        # Log match
        row = OrderedDict()
        row['date'] = date.isoformat()
        row['competition'] = competition
        row['home_team'] = home_team
        row['away_team'] = away_team
        row['home_goals'] = home_goals
        row['away_goals'] = away_goals
        row['home_before'] = round(home_rating, 2)
        row['away_before'] = round(away_rating, 2)
        row['penalty_home'] = 0
        row['penalty_away'] = 0
        row['exp_home'] = round(exp_home, 4)
        row['exp_away'] = round(exp_away, 4)
        row['delta_home'] = round(delta_home, 2)
        row['delta_away'] = round(delta_away, 2)
        row['home_after'] = round(new_home, 2)
        row['away_after'] = round(new_away, 2)

        self.history.append(row)

        # Commit
        self.ratings[home_team] = new_home
        self.ratings[away_team] = new_away
        self.last_active[home_team] = date
        self.last_active[away_team] = date


    def write_outputs(self):
        if not self.history:
            return
        keys = list(self.history[0].keys())
        with open(OUT_TIMESERIES, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=keys)
            writer.writeheader()
            for r in self.history:
                writer.writerow(r)
        with open(OUT_FINAL, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['team', 'rating'])
            for team, r in sorted(self.ratings.items(), key=lambda x: -x[1]):
                writer.writerow([team, round(r, 2)])




In [4]:
# ------------------- Main -------------------
def parse_date(date_str):
    return datetime.datetime.strptime(date_str, "%Y-%m-%d").date()


def read_matches(path):
    matches = []
    with open(path, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            date = parse_date(row['date'])
            home = row['equipe_dom']
            away = row['equipe_ext']
            hg = int(row['buts_dom'])
            ag = int(row['buts_ext'])
            comp = row.get('competition', 'default')
            matches.append((date, home, away, hg, ag, comp))
    matches.sort(key=lambda x: x[0])
    return matches


def main():
    try:
        matches = read_matches(MATCHES_CSV)
    except FileNotFoundError:
        print(f'Error: {MATCHES_CSV} not found')
        sys.exit(1)
    elo = EloRatings()
    for (date, home, away, hg, ag, comp) in matches:
        elo.apply_global_inactivity(date)
        elo.update_match(date, home, away, hg, ag, comp)



    elo.write_outputs()

if __name__ == '__main__':
    main()